# Noções de IA - Trabalho 3 - Rede Convolucional e Transfer Learning
##### Integrantes:
- Lucas Corrêa Boaventura           - 211038262
- Alexandre Castanho Sampaio Rocha  - 211027536

## Introdução
Neste trabalho tentaremos resolver um problema de classificação de imagens coloridas. Para isso utilizaremos uma rede convolucional já treinada para reconhecer imagens utilizando o método do 'fine tuning', ou seja, treinando apenas as camadas finais para classificar as flores do tensorflow_datasets.

### Bibliotecas

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

### Banco de Dados de Flores


In [2]:
(train_ds, test_ds) = tfds.load(
    'tf_flowers',
    split=['train[:70%]', 'train[70%:]'],
    as_supervised=True
)

train_ds, train_labels = tuple(zip(*train_ds))
test_ds, test_labels = tuple(zip(*test_ds))
L = [tf.image.resize(tf.convert_to_tensor(x), (150, 150)) for x in train_ds]
train_ds = tf.convert_to_tensor(L)
L = [tf.image.resize(tf.convert_to_tensor(x), (150, 150)) for x in test_ds]
test_ds = tf.convert_to_tensor(L)

In [3]:
## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))
## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)
print(train_ds.shape)
print(test_ds.shape)

(2569, 150, 150, 3)
(1101, 150, 150, 3)


### Perpectron Multi Camada

#### Para fins de comparação, será treinada um MLP para tentar resolver nosso problema.

In [4]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


### Treinamento com parada prematura

In [5]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 2s 94ms/step - loss: 4010.5947 - accuracy: 0.2448 - val_loss: 877.4156 - val_accuracy: 0.1887
Epoch 2/50
17/17 [==============================] - 1s 85ms/step - loss: 721.2186 - accuracy: 0.3207 - val_loss: 794.5063 - val_accuracy: 0.2568
Epoch 3/50
17/17 [==============================] - 1s 82ms/step - loss: 826.5142 - accuracy: 0.3270 - val_loss: 537.4162 - val_accuracy: 0.2879
Epoch 4/50
17/17 [==============================] - 1s 84ms/step - loss: 435.1420 - accuracy: 0.3470 - val_loss: 306.7443 - val_accuracy: 0.3911
Epoch 5/50
17/17 [==============================] - 1s 79ms/step - loss: 304.2027 - accuracy: 0.3937 - val_loss: 448.9695 - val_accuracy: 0.3327
Epoch 6/50
17/17 [==============================] - 1s 80ms/step - loss: 431.5532 - accuracy: 0.3762 - val_loss: 500.4220 - val_accuracy: 0.3405
Epoch 7/50
17/17 [==============================] - 1s 82ms/step - loss: 266.4644 - accuracy: 0.4414 - val_loss: 523.6260 - val_a

### Avaliação do modelo

In [6]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 0s 6ms/step - loss: 240.0853 - accuracy: 0.4378
Accuracy: 0.4377838373184204


### Análise
Percebe que o MLP não consegue resolver nosso problema de identificação de flores.
Portanto, será aplicado o método transferLearning, em que utilizaremos um modelo convulacional com os pesos já treinados para reconhecimento de imagens.
Também testaremos algumas formas de regularização diferentes.

## Modelo VGG

In [7]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights
## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

### VGG com EarlyStopping 

#### Utilização dos pesos do VGG para a convolução do nosso modelo

In [8]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [9]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 68s 4s/step - loss: 4.3502 - accuracy: 0.5883 - val_loss: 1.4408 - val_accuracy: 0.7160
Epoch 2/50
17/17 [==============================] - 68s 4s/step - loss: 0.5397 - accuracy: 0.8623 - val_loss: 1.3323 - val_accuracy: 0.7335
Epoch 3/50
17/17 [==============================] - 68s 4s/step - loss: 0.1362 - accuracy: 0.9499 - val_loss: 1.3833 - val_accuracy: 0.7588
Epoch 4/50
17/17 [==============================] - 69s 4s/step - loss: 0.0415 - accuracy: 0.9873 - val_loss: 1.3422 - val_accuracy: 0.7510
Epoch 5/50
17/17 [==============================] - 70s 4s/step - loss: 0.0144 - accuracy: 0.9981 - val_loss: 1.3169 - val_accuracy: 0.7626
Epoch 6/50
17/17 [==============================] - 68s 4s/step - loss: 0.0083 - accuracy: 0.9995 - val_loss: 1.2628 - val_accuracy: 0.7646
Epoch 7/50
17/17 [==============================] - 68s 4s/step - loss: 0.0046 - accuracy: 0.9995 - val_loss: 1.2621 - val_accuracy: 0.7802
Epoch 8/50
17/17 [==

#### Avaliação

In [10]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 31s 871ms/step - loss: 1.2847 - accuracy: 0.8138
Accuracy: 0.8138056397438049


### VGG com Dropout

In [11]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    tf.keras.layers.Dropout(0.5),
    dense_layer_2,
    tf.keras.layers.Dropout(0.5),
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

#### Treinamento

In [12]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128)

Epoch 1/50
17/17 [==============================] - 72s 4s/step - loss: 10.3446 - accuracy: 0.3523 - val_loss: 1.1704 - val_accuracy: 0.6323
Epoch 2/50
17/17 [==============================] - 71s 4s/step - loss: 3.0211 - accuracy: 0.4263 - val_loss: 1.0330 - val_accuracy: 0.5875
Epoch 3/50
17/17 [==============================] - 71s 4s/step - loss: 2.1532 - accuracy: 0.4287 - val_loss: 1.1026 - val_accuracy: 0.5584
Epoch 4/50
17/17 [==============================] - 71s 4s/step - loss: 1.7008 - accuracy: 0.4783 - val_loss: 1.0346 - val_accuracy: 0.5778
Epoch 5/50
17/17 [==============================] - 71s 4s/step - loss: 1.6183 - accuracy: 0.5012 - val_loss: 1.0098 - val_accuracy: 0.6148
Epoch 6/50
17/17 [==============================] - 71s 4s/step - loss: 1.3921 - accuracy: 0.5304 - val_loss: 0.9875 - val_accuracy: 0.6401
Epoch 7/50
17/17 [==============================] - 71s 4s/step - loss: 1.3412 - accuracy: 0.5572 - val_loss: 0.9225 - val_accuracy: 0.6537
Epoch 8/50
17/17 [=

#### Avaliação

In [13]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 30s 866ms/step - loss: 0.8459 - accuracy: 0.8065
Accuracy: 0.8065395355224609


### VGG com regularização L1

In [14]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu', kernel_regularizer='l1')
dense_layer_2 = layers.Dense(64, activation='relu', kernel_regularizer='l1')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

#### Treinamento

In [15]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128)

Epoch 1/50
17/17 [==============================] - 70s 4s/step - loss: 124.8422 - accuracy: 0.6024 - val_loss: 90.0386 - val_accuracy: 0.7276
Epoch 2/50
17/17 [==============================] - 68s 4s/step - loss: 69.8569 - accuracy: 0.8730 - val_loss: 50.9012 - val_accuracy: 0.7957
Epoch 3/50
17/17 [==============================] - 69s 4s/step - loss: 39.7574 - accuracy: 0.9557 - val_loss: 30.0290 - val_accuracy: 0.7860
Epoch 4/50
17/17 [==============================] - 69s 4s/step - loss: 23.9992 - accuracy: 0.9630 - val_loss: 19.2117 - val_accuracy: 0.8016
Epoch 5/50
17/17 [==============================] - 69s 4s/step - loss: 15.9705 - accuracy: 0.9679 - val_loss: 13.9699 - val_accuracy: 0.8035
Epoch 6/50
17/17 [==============================] - 69s 4s/step - loss: 12.1210 - accuracy: 0.9436 - val_loss: 11.3335 - val_accuracy: 0.7763
Epoch 7/50
17/17 [==============================] - 69s 4s/step - loss: 10.4792 - accuracy: 0.9051 - val_loss: 10.9306 - val_accuracy: 0.7043
Epoch

#### Avaliação

In [16]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 30s 864ms/step - loss: 3.1372 - accuracy: 0.7738
Accuracy: 0.7738419771194458


### VGG com regularização L2

In [17]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu', kernel_regularizer='l2')
dense_layer_2 = layers.Dense(64, activation='relu', kernel_regularizer='l2')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

#### Treinamento

In [18]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128)

Epoch 1/50
17/17 [==============================] - 71s 4s/step - loss: 7.1352 - accuracy: 0.5771 - val_loss: 4.9572 - val_accuracy: 0.7082
Epoch 2/50
17/17 [==============================] - 70s 4s/step - loss: 3.8214 - accuracy: 0.8545 - val_loss: 4.4898 - val_accuracy: 0.7704
Epoch 3/50
17/17 [==============================] - 71s 4s/step - loss: 3.2887 - accuracy: 0.9562 - val_loss: 4.4601 - val_accuracy: 0.7685
Epoch 4/50
17/17 [==============================] - 70s 4s/step - loss: 3.0122 - accuracy: 0.9859 - val_loss: 4.2499 - val_accuracy: 0.7860
Epoch 5/50
17/17 [==============================] - 70s 4s/step - loss: 2.7886 - accuracy: 0.9971 - val_loss: 3.9427 - val_accuracy: 0.7899
Epoch 6/50
17/17 [==============================] - 70s 4s/step - loss: 2.5939 - accuracy: 1.0000 - val_loss: 3.7639 - val_accuracy: 0.7879
Epoch 7/50
17/17 [==============================] - 70s 4s/step - loss: 2.4155 - accuracy: 1.0000 - val_loss: 3.5994 - val_accuracy: 0.7879
Epoch 8/50
17/17 [==

#### Avaliação

In [19]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 30s 858ms/step - loss: 1.0117 - accuracy: 0.8356
Accuracy: 0.8356040120124817


## Modelo ResNet152

In [20]:
## Loading resnet model
base_model = tf.keras.applications.resnet.ResNet152(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights
## Preprocessing input
train_ds = tf.keras.applications.resnet.preprocess_input(train_ds)
test_ds = tf.keras.applications.resnet.preprocess_input(test_ds)
base_model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 75, 75, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                      

### ResNet com EarlyStopping

In [21]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [22]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 102s 6s/step - loss: 5.4257 - accuracy: 0.5192 - val_loss: 1.1291 - val_accuracy: 0.7568
Epoch 2/50
17/17 [==============================] - 95s 6s/step - loss: 0.6545 - accuracy: 0.8234 - val_loss: 0.8484 - val_accuracy: 0.8113
Epoch 3/50
17/17 [==============================] - 95s 6s/step - loss: 0.2342 - accuracy: 0.9192 - val_loss: 0.9613 - val_accuracy: 0.7957
Epoch 4/50
17/17 [==============================] - 95s 6s/step - loss: 0.0948 - accuracy: 0.9655 - val_loss: 0.7570 - val_accuracy: 0.8249
Epoch 5/50
17/17 [==============================] - 95s 6s/step - loss: 0.0287 - accuracy: 0.9932 - val_loss: 0.8012 - val_accuracy: 0.8210
Epoch 6/50
17/17 [==============================] - 95s 6s/step - loss: 0.0309 - accuracy: 0.9917 - val_loss: 0.9222 - val_accuracy: 0.8016
Epoch 7/50
17/17 [==============================] - 95s 6s/step - loss: 0.0172 - accuracy: 0.9981 - val_loss: 0.8522 - val_accuracy: 0.8307
Epoch 8/50
17/17 [=

#### Avaliação

In [23]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 44s 1s/step - loss: 0.8316 - accuracy: 0.8193
Accuracy: 0.8192552328109741


### ResNet152 com Dropout

In [24]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    tf.keras.layers.Dropout(0.5),
    dense_layer_2,
    tf.keras.layers.Dropout(0.5),
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

#### Treinamento

In [25]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128)

Epoch 1/50
17/17 [==============================] - 105s 6s/step - loss: 4.3070 - accuracy: 0.3144 - val_loss: 1.3255 - val_accuracy: 0.3969
Epoch 2/50
17/17 [==============================] - 99s 6s/step - loss: 1.6699 - accuracy: 0.3012 - val_loss: 1.2973 - val_accuracy: 0.3930
Epoch 3/50
17/17 [==============================] - 99s 6s/step - loss: 1.5551 - accuracy: 0.3236 - val_loss: 1.1935 - val_accuracy: 0.4125
Epoch 4/50
17/17 [==============================] - 99s 6s/step - loss: 1.5138 - accuracy: 0.3440 - val_loss: 1.1273 - val_accuracy: 0.4942
Epoch 5/50
17/17 [==============================] - 99s 6s/step - loss: 1.3571 - accuracy: 0.3898 - val_loss: 1.0576 - val_accuracy: 0.5311
Epoch 6/50
17/17 [==============================] - 99s 6s/step - loss: 1.2812 - accuracy: 0.4190 - val_loss: 1.0609 - val_accuracy: 0.5953
Epoch 7/50
17/17 [==============================] - 99s 6s/step - loss: 1.2865 - accuracy: 0.4360 - val_loss: 1.1106 - val_accuracy: 0.5195
Epoch 8/50
17/17 [=

#### Avaliação

In [26]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 44s 1s/step - loss: 0.9042 - accuracy: 0.7720
Accuracy: 0.7720254063606262


## Conclusão
Pelas avaliações obtidas, obtivemos um resultado com a accuracy variando em torno de 80%. Levando em conta que esse banco de dados é difícil, por apresentar imagens que seres humanos dificilmente conseguiriam identificar, como foto de um jardim com um flor no canto, esse resultado é satisfatório.